# Imports

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

---

# NMA Provided Codes

## Dataset Information

The HCP dataset comprises task-based fMRI from a large sample of human subjects. The NMA-curated dataset includes time series data that has been preprocessed and spatially-downsampled by aggregating within 360 regions of interest.

In [2]:
# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 100

# The data have already been aggregated into ROIs from the Glasser parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in seconds

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated twice in each subject
RUNS   = ['LR','RL']
N_RUNS = 2

# There are 7 tasks. Each has a number of 'conditions'
# TIP: look inside the data folders for more fine-graned conditions

EXPERIMENTS = {
    'MOTOR'      : {'cond':['lf','rf','lh','rh','t','cue']},
    'WM'         : {'cond':['0bk_body','0bk_faces','0bk_places','0bk_tools','2bk_body','2bk_faces','2bk_places','2bk_tools']},
    'EMOTION'    : {'cond':['fear','neut']},
    'GAMBLING'   : {'cond':['loss','win']},
    'LANGUAGE'   : {'cond':['math','story']},
    'RELATIONAL' : {'cond':['match','relation']},
    'SOCIAL'     : {'cond':['ment','rnd']}
}

subjects = np.loadtxt('subjects_list.txt', dtype='str')

## Understanding the folder organisation

The data folder has the following organisation:

- hcp
  - regions.npy (information on the brain parcellation)
  - subjects_list.txt (list of subject IDs)
  - subjects (main data folder)
    - [subjectID] (subject-specific subfolder)
      - EXPERIMENT (one folder per experiment)
        - RUN (one folder per run)
          - data.npy (the parcellated time series data)
          - EVs (EVs folder)
            - [ev1.txt] (one file per condition)
            - [ev2.txt]
            - Stats.txt (behavioural data [where available] - averaged per run)
            - Sync.txt (ignore this file)

## Region information

In [3]:
regions = np.load(f"regions.npy").T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    hemi=['Right']*int(N_PARCELS/2) + ['Left']*int(N_PARCELS/2),
)

## Helpers

In [4]:
def load_single_timeseries(subject, experiment, run, remove_mean=True):
  """Load timeseries data for a single subject and single run.

  Args:
    subject (str):      subject ID to load
    experiment (str):   Name of experiment
    run (int):          (0 or 1)
    remove_mean (bool): If True, subtract the parcel-wise mean (typically the mean BOLD signal is not of interest)

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_run  = RUNS[run]
  bold_path = f"subjects/{subject}/{experiment}/tfMRI_{experiment}_{bold_run}"
  bold_file = "data.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)
  return ts


def load_evs(subject, experiment, run):
  """Load EVs (explanatory variables) data for one task experiment.

  Args:
    subject (str): subject ID to load
    experiment (str) : Name of experiment
    run (int): 0 or 1

  Returns
    evs (list of lists): A list of frames associated with each condition

  """
  frames_list = []
  task_key = f'tfMRI_{experiment}_{RUNS[run]}'
  for cond in EXPERIMENTS[experiment]['cond']:
    ev_file  = f"subjects/{subject}/{experiment}/{task_key}/EVs/{cond}.txt"
    ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
    ev       = dict(zip(["onset", "duration", "amplitude"], ev_array))
    # Determine when trial starts, rounded down
    start = np.floor(ev["onset"] / TR).astype(int)
    # Use trial duration to determine how many frames to include for trial
    duration = np.ceil(ev["duration"] / TR).astype(int)
    # Take the range of frames that correspond to this specific trial
    frames = [s + np.arange(0, d) for s, d in zip(start, duration)]
    frames_list.append(frames)

  return frames_list

---

# Analysis

## Define Areas

In [6]:
"""
Note that these regions are present in the separately for right and left
So use R_ or L_ before these to get the necessary data
Define labels for areas
"""
DLPFC_LABELS = ["8C", "8Av", "i6-8", "s6-8", "SFL", "8BL", "9p", "9a", "8Ad", "p9-46v", "a9-46v", "46", "9-46d"]
PFC_LABELS = ['R_'+i for i in DLPFC_LABELS]
PFC_LABELS += ['L_'+i for i in DLPFC_LABELS]
FFC_LABELS = ['R_FFC', 'L_FFC']
BODY_LABELS = ['R_VVC', 'L_VVC']
PARA_LABELS = ["PHA3", "STSda", "V4t"]
PPA_LABELS = ['R_'+i for i in PARA_LABELS]
PPA_LABELS += ['L_'+i for i in PARA_LABELS]
LOC_LABELS = ['R_VMV2', 'L_VMV2']

## Some more helper methods

In [21]:
def get_area_data(data, area):
    """Returns the data for a given area label"""
    area_idx = np.where(regions[0] == area)[0] # Get area indices
    return data[area_idx, :] # Get that area from the data and return

def get_area_mean(data, areas):
    mean = []
    for area in areas:
        mean.append(get_area_data(data, area))
    return np.array(mean).mean(axis=0)

def get_ev_indices(exp, cond):
    """Returns the condition (time) indices relating to fMRI data"""
    return EXPERIMENTS[exp]['cond'].index(cond)

def get_cond_related_signal(data, evs, idx):
    return data[:, evs[idx]]

def get_all_subject_data(exp, area_labels, cond, remove_first_n=0, take_mean=False):
    """
    If take_mean, mean is taken across all subjects, so a single timeseries is returned.
    """
    all_data = []
    for s in subjects: # For all subjects
        for r in [0, 1]: # For all runs
            # Get data and event indices
            data = load_single_timeseries(subject=s, experiment=exp,
                                          run=r, remove_mean=True)
            evs = load_evs(subject=s, experiment=exp,run=r)
            
            idx = get_ev_indices(exp, cond)
            
            area_data = get_area_mean(data, area_labels)
            cond_data = get_cond_related_signal(area_data, evs, idx)[0][0][remove_first_n:]
            all_data.append(cond_data)
    
    all_data = np.array(all_data)
    if take_mean:
        return all_data.mean(axis=0)
    else:
        return all_data

For example to get FFC region for 2 back faces task in working memory experiment while also removing the first 4 TRs (for instruction phase reasons), we can use the method below. From the length, we can see we got 200; 100 subjects with 2 runs each. 

In [20]:
get_all_subject_data('WM', FFC_LABELS, '2bk_faces', remove_first_n=4).shape

(200, 35)

We could use take_mean=True to get the mean signal across subjects. This should give length of 35 since there are 39 TRs but we are removing the first 4.

In [23]:
get_all_subject_data('WM', FFC_LABELS, '2bk_faces', remove_first_n=4, take_mean=True).shape

(35,)